In [1]:
# default_exp restricted

In [2]:
#hide
from nbdev.showdoc import *

In [3]:
#export
import matplotlib as mpl
import numpy as np
from os import path, makedirs
from copy import copy
from math import factorial
from matplotlib import pyplot as plt
from numpy.random import rand as rnd
from numpy import newaxis as nax
from numpy import real_if_close as ric
from scipy.optimize import brentq
from sys import exit as se, argv

# Restricted Hartree-Fock
> API details. author: Lorenzo Cardarelli

## Mean-field Hamiltonian in k-space, 8-sites cell


This part of the library focuses on solving the system Hamiltonian
\begin{aligned}
H = &-t_0\sum_{<ij>}(c_i^\dagger c_j + H.c.)+J^A_x\sum_{i\in A}(c_{i}^\dagger c_{i+x} + H.c.) 
        +J^A_y\sum_{i\in A}(c_{i}^\dagger c_{i+y} + H.c.)\\
        &+J^B_x\sum_{i\in B}(c_{i}^\dagger c_{i+x} + H.c.) +J^B_y\sum_{i\in B}(c_{i}^\dagger c_{i+y} + H.c.) 
        +V_1\sum_{<ij>}n_in_j+V_2\sum_{<<ij>>}n_in_j\\
        &+V_3\sum_{<ij>_3}n_in_j+V_4\sum_{<ij>_4}n_in_j
\end{aligned}   
self-consistently with a restricted Hartree fock ansatz on the 8-site unit cell.

In [4]:
#export
class checkerboard_eight_sites():
    def __init__(self):
        # 8-sites checkerboard attributes
        self.classtag = 'checkerboard_eight_sites'
        self.site_coos = np.array([[0,2,1,3,0,2,1,3],[0,0,1,1,2,2,3,3]]).T   
        self.cellBravaisvecs = np.array([[4.,0.], [0.,4.]])
        self.pairs_negative_J_flux = ([0,1,2,6,4,5,3,7],[4,5,3,7,0,1,2,6])
        self.nsites = 8 # Number of sites in the unit cell.
        self.ncs = 5 # Min number of neighboring cells (+1) needed for lattice tiling.
        self.xiQAH_loop_mask = (np.array([1,2,5,3]), np.array([2,5,3,1])) # Coordinates of the sites on a square plaquette of the unit cell over which the topological order parameter is calculated.
        # Checkerboard attributes and methods
        self.set_lattice_specs()
        # Set that lattice is 2-dimensional
        self.d = 2
        self.get_kgrid()
        self.get_eikd()
        return

    def get_hopping_matrix(self):
        hop = np.zeros([self.nV,5,self.nBands,self.nBands])
        hop[1] = -self.t*self.distance_mask_hc[1]
        m = np.ones([self.nsites,self.nsites],dtype=int)
        m[self.pairs_negative_J_flux] = -1
        sign_mask = np.ones([5,self.nsites,self.nsites])*m
        hop[2] = self.J*self.distance_mask_hc[2]*sign_mask
        for _ in range(3,self.nV):
            hop[_] = hop[0]*0
        self.bare_hopping_hc = hop
        return

    def init_EkUk(self):
        shape = list(self.kgridsize)
        n = self.nBands
        self.Ek = np.zeros(shape + [n], dtype=float)
        self.Uk = np.zeros(shape + [n, n], dtype=complex)
        return

    def get_kuv(self):
        # k-lattice unit vectors
        k1 = 2*np.pi*np.array([1.,0.])
        k2 = 2*np.pi*np.array([0.,1.])
        return k1, k2

    def set_lattice_specs(self):
        e1 = [1,-1]; e2 = [1,1]; # Bravais vectors
        self.square_distances = [0,2,4,8,10,18][:self.nV] # square of the distances between lattice sites
        self.r_prim_vecs = np.array([e1,e2])
        self.k_prim_vecs = np.pi * self.r_prim_vecs / np.linalg.norm(e1)**2
        self.FTvecs = self.r_prim_vecs
        self.nBands = self.site_coos.shape[0]
        self.band_filling = np.rint(self.nBands/2).astype(int) # half-filling
        return

    def get_interaction_matrix(self):
        self.interactions_hc = np.zeros([self.nV,5,self.nBands,self.nBands])
        for _ in range(self.nV):
            self.interactions_hc[_] = self.Vsarray[_]*self.distance_mask_hc[_]
        returncla

    def get_Hoffdiag(self):
        bx, by = self.BZindices
        H = self.bare_hopping_hc*self.eikdhc[:,bx,by]
        H -= self.interactions_hc*self.eikdhc[:,bx,by]*np.conj(self.corrhc[:])
        H = np.sum(H,axis=(0,1))
        return H

    def get_Hdiag(self):
        nmat = np.ones([self.nBands,self.nBands])*np.diag(self.corrhc[0,0,:,:])
        mfs = np.sum( nmat*self.distance_weight_hc_flat, axis=2)
        H = np.diag( np.sum(self.Vsarray[:,nax]*mfs, axis=0))
        return H

    def get_Hc(self, d=0):
        d = np.diag( ric(self.corr[0,0,:,:]) )
        nmat = np.ones([self.nBands,self.nBands]) * d
        Hc = 0
        Hc = -np.sum( self.Vsarray[:,nax,nax]*self.distance_weight_flat*nmat*nmat.T )
        Hc += np.sum( self.Vsarray[:,nax,nax,nax]*self.cosq )
        Hc = ric(Hc)
        return Hc

    def get_Hamiltonian(self):
        self.H = self.get_Hoffdiag()
        self.H += self.get_Hdiag()
        return

    def init_outputdata(self):
        self.basic_params = ['rho%d' %_ for _ in range(self.nV)] + ['xiQAH']
        MFp = {}
        for key in self.basic_params:
            MFp[key] = rnd()
        self.outputdata = copy(MFp)
        self.oldoutputdata = copy(MFp)
        self.osc_counter = 0
        return

    def init_rnd_corr(self,verbose=True):
        # corrhc : [neighood,cp,nB,nB]
        def foo(k=0):
            return rnd( self.nV-k, 5, self.nBands, self.nBands ).astype(np.complex128)
        corrhc = 0*foo()
        if '-corrm' in argv:
            if self.corrm == 'real':
                corrhc[1:] = foo(1)
                self.permaprt('Correlation matrix initialized to random real values.',verbose=verbose)
            elif self.corrm == 'comp':
                corrhc[1:] = foo(1) + 1.j*foo(1)
                self.permaprt('Correlation matrix initialized to random complex values.',verbose=verbose)
            else:
                se('To initialize the correlation matrix to real (complex) values, please use "-corrm real" (comp).')
        else:
            corrhc[1:] = foo(1) + 1.j*foo(1)
            self.permaprt('Correlation matrix initialized to random complex values.',verbose=verbose)
        d = np.random.rand(self.nBands)
        d *= self.band_filling/np.linalg.norm(d,1) 
        np.fill_diagonal(corrhc[0,0], d)
        self.corrhc = corrhc*self.distance_mask_hc
        self.get_corr_cosq()
        return

    # ********** Correlations, data

    def get_xiQAH(self):
        xiQAH = np.abs(np.sum(np.imag(self.corrhc[1,0][self.xiQAH_loop_mask])))/4
        return xiQAH

    def get_rho(self, d):
        n = ric(np.diag(self.corrhc[0,0]))
        w = np.where( self.distance_mask[d,0] )
        ni = n[w[0]]
        nj = n[w[1]]
        rho = np.prod(np.abs(ni-nj))
        return rho

    def get_outputdata(self):
        self.outputdata['xiQAH'] = self.get_xiQAH()
        for d in range(0, self.nV):
            self.outputdata['rho%d'%d] = self.get_rho(d)
        return

    def sort(self, w, v):
        lowfirst = np.argsort(w)
        w = w[lowfirst]
        v = v[:, lowfirst]
        return w,v

    def check_convergence(self,verbose=False):
        self.oldcorr_grad_max = copy(self.corr_grad_max)
        grad = self.oldcorrhc - self.corrhc
        agrad = np.abs(grad)
        amax = np.max(agrad)
        self.corr_grad_max = grad[np.where(agrad == amax)][0]
        self.delta_FE = abs(self.oldFE - self.FE)
        cgd_norm = np.linalg.norm(self.corr_grad_max)
        if verbose:
            self.permaprt('self.FE : %.16f, correlation gradient norm : %.16f' %(self.FE, cgd_norm))
        # If the max gradient is above 1.E-4 for more than 100 iters, call it oscillation.
        if cgd_norm >= self.oscillation_threshold:
            self.large_gradient_counter += 1
            if self.large_gradient_counter == self.consecutive_oscillations:
                self.terminate = True
        else:
            self.large_gradient_counter += 1
        # Convergence if both cgd_norm and delta_FE below threshold or if one mean-field operator is self.convergence_thresh_mfops times larger than all others.
        if (max(cgd_norm, self.delta_FE) < self.convergence_thresh_cFE) or self.mf_ops_converged():
            self.converged = True
        else:
            self.converged = False
        return

    def set_fpmethod_convergence_params(self):
        self.iterations = 0
        self.corr_grad_max = 10000
        self.converged = False
        self.terminate = False
        self.large_gradient_counter = 0
        return

    def fixed_point_method(self):
        for self.iterations in range(self.itermax):
            self.diagH()
            self.get_correlation_tensors()
            self.get_FE()
            self.update_outputdata()
            self.check_convergence()
            if self.iterations > self.itermin and (self.converged or self.terminate):
                return self.converged, self.terminate
        return False, False

    def prt_outputdata(self):
        for key in self.basic_params:
            v = self.outputdata[key]
            self.permaprt('%s ' %key, 0 if v<self.convergence_thresh_mfops else v)
        return

    def prt_postrun_data(self,verbose=True):
        if verbose:
            self.permaprt('Run concluded.\nFinal number of iterations: %d out of %d.' %(self.iterations,self.itermax))
            self.permaprt('Correlation gradient: %.0e' %np.linalg.norm(self.corr_grad_max))
            self.permaprt('Free energy gradient: %.0e' %self.delta_FE)
            self.permaprt('Ratio two largest order parameters: %.0e' %self.mf_ops_gap)
            self.prt_outputdata()
            self.permaprt('')
        return

    def intBZ(self, integrand):
        if len(integrand.shape) == 1:
            tup = (0)
        else:
            tup = (0,1)
        return np.real_if_close( np.sum(integrand, axis=tup) / np.prod(self.kgridsize) )

    def get_vars(self):
        nB = self.nBands # number of particles in the cell : 4/8
        h = int(nB/2)
        N = int(factorial(nB)/factorial(h)**2) # combinations w/o repetitions : 6/70
        neighood = len(self.square_distances) # number of square distances considered : 3/4
        Bv = self.d # Bravais vectors
        xy = self.d # spatial coordinates
        pair = 2 # number of elements in a pair (of unit cells)
        cp = 5 # cells paired
        nkx, nky = self.kgridsize
        return xy,nB,h,N,neighood,pair,Bv,cp,nkx,nky

    def get_corr_cosq(self):
        self.corr = self.corrhc*self.distance_mask
        self.cosq = np.multiply(self.corr, np.conj(self.corr))
        self.cosqhc = np.multiply(self.corrhc, np.conj(self.corrhc))
        return

    def get_correlation_tensors(self):
        # Cd, C, bd, b : C/b are cell-/Bogoliubov-operators.
        xy,nB,h,N,nhood,pair,Bv,cp,nkx,nky = self.get_vars()

        U = np.conj(self.Uk) # [nkx, nky, Cd, bd]

        beta = self.beta
        energies_fermi = np.copy(self.Ek)
        [a,b] = [np.amin(energies_fermi), np.amax(energies_fermi)]
        def numbern(mu_, nn_, beta_, energies_):
            e_aux = np.copy(energies_)
            e_aux[beta_ * (e_aux - mu_) > 30] = 30 / beta_ + mu_
            ferm = 1 / (np.exp(beta_ * (e_aux - mu_)) + 1)
            return np.sum(ferm) - nn_
        nn = self.band_filling*np.prod(self.kgridsize)
        self.mu = brentq(numbern, a, b, args=(nn, beta, energies_fermi))
        energies_fermi[beta*(energies_fermi - self.mu) > 30] = 30./beta + self.mu
        fermi_distribution = 1. / (np.exp(beta * (energies_fermi - self.mu)) + 1)
        fermi_distribution = self.repeat(fermi_distribution, nB).transpose(0,1,3,2)

        U *= np.sqrt(fermi_distribution[:,:,:,:])
        U = self.repeat(U, nB, nB) # [nkx, nky, Cd, bd, rep, rep]
        Ud = np.conj(U.transpose(0,1,4,5,2,3)) # [nkx, nky, rep, rep, C, b]
        UUd = np.multiply(U,Ud) # [nkx, nky, Cd, bd, C, b]
        UUd = np.trace(UUd, axis1=3, axis2=5) # [nkx, nky, Cd, C]

        newcorrhc = np.zeros([nhood,cp,nB,nB], dtype=np.complex128)
        for _ in range(nhood):
            newcorrhc[_] = self.intBZ(UUd[:,:,nax,:,:,]*self.eikdhc[_])
        self.oldcorrhc = np.copy(self.corrhc)
        self.corrhc = newcorrhc
        self.get_corr_cosq()
        return

    def calc_correlations(self):
        n = self.nBands
        m = int(n/2) # Half filling set here.
        km = self.kw.shape[2]
        U = self.repeat(self.Uk, n, n)
        Ud = np.conj(U.transpose(0,1,4,5,2,3))
        UUd = np.multiply(U,Ud)
        corr = np.trace(UUd[:,:,:,:m,:,:m], axis1=3, axis2=5) 
        corr = self.repeat(corr, km)
        kweights = self.kw.transpose(3,4,0,1,2) # (0,1,3,4,2)
        corr = np.multiply(corr, kweights)
        corr = self.intBZ(corr)
        corr = np.divide(corr, np.atleast_3d(self.nb))
        self.corr = corr
        self.cosq = np.multiply(corr, np.conj(corr))
        return

    def repeat(self, array, *args):
        shape = list(array.shape) + list(args)
        return array.repeat(np.prod(args)).reshape(shape)


    def diagH(self):
        Nkx, Nky = self.kgridsize
        self.oldEk = np.copy(self.Ek)
        for ix in range(Nkx):
            for jy in range(Nky):
                self.BZindices = (ix, jy)
                self.get_Hamiltonian()
                w,v = np.linalg.eigh(self.H) # w[i] i-th e.value, v[:,i] i-th column e.vector
                w,v = self.sort(w,v) # Sort the eigenvalues/vectors in increasing order of evalues.
                self.Ek[ix,jy] = np.real_if_close(w)
                self.Uk[ix,jy] = np.real_if_close(v)
        return

    def get_kgrid(self):
        nkx, nky = self.kgridsize
        k1 = self.k_prim_vecs[0]
        k2 = self.k_prim_vecs[1]
        ks1 = np.reshape( np.kron(k1,np.linspace(0,1,nkx,endpoint=False)), (2,-1))
        ks2 = np.reshape( np.kron(k2,np.linspace(0,1,nky,endpoint=False)), (2,-1))
        # Repeat to promote linear ks to 2-d grids
        ks1 = ks1.repeat(nky).reshape(self.d,nkx,nky).transpose(1,2,0)
        ks2 = ks2.repeat(nkx).reshape(self.d,nky,nkx).transpose(2,1,0)
        # Sum the two to get_ kgrid
        # kgrid: tensor with shape [nkx,nky,xy]
        self.kgrid = ks1 + ks2
        return

    def update_outputdata(self):
        for key in self.outputdata.keys():
            self.oldoutputdata[key] = self.outputdata[key]
        self.get_outputdata()
        return

    def get_FE(self):
        Hc = self.get_Hc()
        bint = ric(np.sum(self.intBZ(self.Ek[:,:,:self.band_filling])))
        self.oldFE = copy(self.FE)
        self.FE = Hc + bint
        return self.FE

    def get_eikd(self):
        # site_coos : [nB,xy] 
        # cellBravaisvecs : [Bv,xy]
        xy,nB,h,N,neighood,pair,Bv,cp,nkx,nky = self.get_vars()
        cpc = np.array([[[0,0],[0,0],[0,0],[0,0],[0,0]], [[0,0],[0,1],[1,1],[1,0],[1,-1]]]) # [pair,cp,Bv]
        pair_coos = np.tensordot(cpc, self.cellBravaisvecs, axes=(2,0)) # [pair,cp,xy]
        site_coos = self.repeat(self.site_coos, nB, cp, pair).transpose(3,0,2,1,4) # [cp,nB,nB,xy,pair]
        site_coos[:,:,:,:,1] = site_coos[:,:,:,:,1].transpose(0,2,1,3)
        cell_pair_coos = self.repeat(pair_coos, nB, nB).transpose(1,3,4,2,0) # [cp,nB,nB,xy,pair]
        pair_distance = np.squeeze(np.diff(site_coos + cell_pair_coos, axis=4)) # [cp,nB,nB,xy] check this
        distance_mat = np.sum(pair_distance**2, axis=3) # [cp,nB,nB]

        distance_mask = np.zeros([neighood,cp,nB,nB], dtype=int) # [neighood,cp,nB,nB]
        for _ in range(neighood):
            distance_mask[_][np.where(distance_mat==self.square_distances[_])] = 1
        distance_mask[:,0] *= np.triu(np.ones([nB,nB],dtype=int))[nax,:,:] # not ancillary, cures off-diag on cell-0

        # distance_mask : [neighood,cp,nB,nB]
        # distance_weight_flat : [neighood,nB,nB]
        # distance_mask_flat : [neighood,nB,nB]
        def foo(mat):
            mat[mat!=0] = 1
            return mat
        def foo2(mat):
            sh = mat.shape
            m = np.zeros(sh, dtype=int)
            m[:,:] = np.eye(sh[-1])
            return 1-m

        # *** triangular masks
        self.distance_mask = np.copy(distance_mask)
        self.distance_weight_flat = np.sum(self.distance_mask, axis=1)
        self.distance_mask_flat = foo(np.copy(self.distance_weight_flat))
        # *** Hermitian conjugate masks
        self.distance_mask_hc = distance_mask + distance_mask.transpose(0,1,3,2)*foo2(distance_mask)
        self.distance_weight_hc_flat = np.sum( self.distance_mask_hc, axis=1)
        self.distance_mask_hc_flat = foo( np.copy( self.distance_weight_hc_flat))

        # eikd : [neighood,nkx,nky,cp,nB,nB]
        kd = np.tensordot(self.kgrid, pair_distance, axes=(2,3)) # [nkx,nky,cp,nB,nB]
        eikd = np.exp(-1.j*kd)
        self.eikd = np.zeros([neighood,nkx,nky,cp,nB,nB], dtype=np.complex128) # [neighood,nkx,nky,cp,nB,nB]
        for _ in range(neighood):
            self.eikd[_] = eikd*distance_mask[_,nax,nax,:,:,:]
        self.eikdhc = self.eikd + np.conj(self.eikd.transpose(0,1,2,3,5,4))*foo2(self.eikd)
        return




Class defining the checkerboard lattice model.

Methods
----
| Method | Description |
|----|----|
|`__init__` |  Initialize lattice related parameters  |
|`fixed_point_method` |  Fixed-point search routine  |
|`get_hamiltonian` |  Define the mean-field Hamiltonian, dependent on mean-field coefficients given by the correlation matrix  |
|`diagH`|  Diagonalize the Hamiltonian for each $k$-vector in the Brilluoin zone. Obtain new k-space eigen-vectors and eigen-values (bands)  |
|`get_correlation_tensors` |  Calculate correlation matrix  |
|`get_FE`|  Calculate free energy  |
|`check_convergence` |  Stop the fixed-point search i) after `itermax` iterations or ii) after `itermin` iterations and if iia) the largest mean-field parameter does not converge but oscillates between $\pm n$ for `consecutive_oscillations` times, indicating a metallic phase (discard run) or iib) if both the free energy gradient and the correlation matrix gradient converged below `convergence_thresh_cFE` or the inverse ratios between the largest mean-field parameter and all others converged below `convergence_thresh_mfops`  |

Attributes
----
| Attribute | type | Description |
|----|-----|----|
| `cellBravaisvecs` | int nparray | Bravais vectors of the unit cell |
| `iterations` | int | Counts iterations |
| `itermin` | int | Minimun number of iterations |
| `itermax` | int | Maximun number of iterations |
| `ncs` | int | Minimum number of neighboring cells (+1) needed to achieve full lattice tiling |
| `nsites` | int | Number of lattice unit cell sites |
| `site_coos` | int nparray | Sites coordinates in the lattice unit cell |
| `Vsarray` | float nparray | Interaction amplitudes, `nV` terms including $V_0$ |
| `xiQAH_loop_mask` | int nparray | Coordinates of the square plaquette over which $\xi_{\textrm{QAH}}$ is calculated |

In [ ]:
# export
class mean_field_analysis(checkerboard_eight_sites):
    def __init__(self, itermin=20, itermax=700, kgridsize=10, Vgrid=5,
                 beta=100, corrm='comp', rydberg=False):

        # Number of interaction terms considered: nV=5 --> {V0, V1, V2, V3, V4}
        self.nV = 5 

        # Parameters.
        self.rydberg = rydberg
        self.kgridsize = [kgridsize, kgridsize] # Grating of the first Brilluoin zone
        self.V1V2region = [2,6,0,4] # Region of V1-V2
        self.Vsgridshape = [1,Vgrid,Vgrid,1,1] # Size of the grid in the region of {V1,V2}
        ''' Run '''
        self.itermin = itermin # Min number of fixed-point search iterations
        self.itermax = itermax # Max number of fixed-point search iterations
        self.beta = beta # Inverse temperature 1/kT
        self.corrm = corrm
        self.convergence_thresh_cFE = 1.E-12 # Threshold on correlation and FE for search convergence
        self.convergence_thresh_mfops = 1.E-4 # Threshold on mf operators for search convergence
        self.consecutive_oscillations = 200 # Consecutive ±1 oscilattions of charge order parameter, signalling non-convergence (metallic phase)
        self.oscillation_threshold = 1.E-4 # Threshold below which oscillation count starts
        ''' Plot '''
        self.dpi = 500
        self.beta = 100 # Inverse temperature store1/kT
        self.format = 'png'
        self.datadir = 'data'

        # Create data and figures directories.
        self.datadir = 'data'
        self.figuresdir = 'figures'
        if not path.exists('./'+self.datadir): makedirs('./'+self.datadir)
        if not path.exists('./'+self.figuresdir): makedirs('./'+self.figuresdir)

        # These parameters must not change.
        self.t = 1.0
        self.J = 0.5

        # Initialize some attributes.
        self.markcol = ['#94cef3', '#6e92e8', '#cccccc', '#636363', '#fdbe85', '#3182bd', '#feedde', '#fb6a4a']
        self.edgecol = ['#4799d1', '#193b78', '#a3a3a3', '#4f4f4f', '#fc9639', '#276897', '#fc9639', '#b5211b']
        self.FE = 5
        self.delta_FE = 5

        # After all relevant parameteres have been defined, initialize the parent class.
        checkerboard_eight_sites.__init__(self)
        return

    def get_empty_Vsdict(self):
        Vsdict = {}
        for _ in range(self.nV):
            Vsdict[_] = 0
        return Vsdict

    def get_pool(self,rydberg=False,fullryd=True):
        Vsarray_pool, Vsarray_pool_ryd, compact_Vsarray_pool_ryd = self.get_Vsarray_pool()
        if rydberg:
            if fullryd:
                return Vsarray_pool_ryd
            else:
                return compact_Vsarray_pool_ryd 
        else:
            return Vsarray_pool

    def update_Vsarray(self,Vsarray,kwargs):
        self.Vsarray = np.zeros(self.nV)
        if 'V1' and 'V2' in kwargs.keys():
            self.Vsarray[1:3] = [kwargs['V1'],kwargs['V2']]
        else:
            if len(Vsarray) == 2:
                self.Vsarray[1:3] = Vsarray
            else:
                self.Vsarray[:len(Vsarray)] = Vsarray
        return

    def prepare_run(self, Vsarray=[0,3,3], verbose=True, rydberg=False, **kwargs):
        self.update_Vsarray(Vsarray, kwargs)
        self.init_EkUk()
        self.get_hopping_matrix()
        self.get_interaction_matrix()
        self.init_outputdata()
        self.permaprt('Running ' + ''.join(['V%s=%.2f, ' %(num,arg) for num,arg in enumerate(self.Vsarray)])[:-2], verbose=verbose)
        self.init_rnd_corr(verbose)
        self.set_fpmethod_convergence_params()
        return

    def prepare_grid_run(self, V3=0, verbose=False, Vgrid=4, V1V2region=[2,6,0,4], rydberg=False):
        # Get pool of Vsarray.
        self.V1V2region = V1V2region # Region of V1-V2
        self.Vsgridshape = [1,Vgrid,Vgrid,1,1] # Size of the grid in the region of {V1,V2}
        self.Vsdict = self.get_empty_Vsdict()
        self.Vsdict[1], self.Vsdict[2] = self.get_V1V2s()
        self.Vsdict[3] = V3
        self.init_datadict()
        return

    def single_run(self, Vsarray=[0,3,3], verbose=True, rydberg=False, **kwargs):
            self.prepare_run(Vsarray=Vsarray, verbose=verbose, rydberg=rydberg, **kwargs)
            V1,V2 = self.Vsarray[1],self.Vsarray[2]
            if not (rydberg and (V2>=V1 or V1>=8*V2)):
                self.fixed_point_method()
                self.prt_postrun_data(verbose)
            return

    def sanity_check(self,rydberg=False):
        V1,V2 = self.Vsarray[1],self.Vsarray[2]
        if not (rydberg and (V2>=V1 or V1>=8*V2)):
            return True
        else:
            return False

    def grid_run(self, V3=0, verbose=False, Vgrid=4, V1V2region=[2,6,0,4], rydberg=False):
        self.prepare_grid_run(V3=V3, verbose=verbose, Vgrid=Vgrid, V1V2region=V1V2region, rydberg=rydberg)
        Vsarray_pool = self.get_pool(rydberg=rydberg)
        counter = 0
        for Vsarray in Vsarray_pool:
            self.prepare_run(rydberg=rydberg, Vsarray=Vsarray, verbose=verbose)
            if self.sanity_check(rydberg=rydberg):
                self.fixed_point_method()
                self.prt_postrun_data(verbose)
            self.store_grid_data(counter)	
            counter += 1
        return

    def store_grid_data(self,counter):
        V2 = self.Vsgridshape[1]
        i,j = counter//V2, counter%V2
        self.datadict['Ek'][0,i,j] = self.Ek
        self.datadict['FE'][0,i,j] = self.FE
        self.datadict['xiQAH'][0,i,j] = self.outputdata['xiQAH']
        self.datadict['corrhc'][0,i,j] = self.corrhc 
        for _ in range(self.nV):
            self.datadict['rho%d'%_][0,i,j] = self.outputdata['rho%d'%_]
        return

    def init_datadict(self):
        Vgs = self.Vsgridshape
        def scal(): return np.zeros( self.Vsgridshape, dtype=float)
        def Ek(): return np.zeros(Vgs + self.kgridsize + [self.nsites], dtype=float)
        def crhc(): return np.zeros( self.Vsgridshape + [self.nV, self.ncs, self.nsites, self.nsites], dtype=complex)
        self.datadict = {
            'Ek' : Ek(),
            'FE' : scal(),
            'xiQAH' : scal(),
            'corrhc' : crhc(),
        }
        for _ in range(self.nV):
            self.datadict['rho%d'%_] = scal()
        return

    def get_V1V2_mat(self):
        N1, N2 = self.Vsgridshape[1], self.Vsgridshape[2]
        V1, V2 = self.Vsdict[1], self.Vsdict[2]
        V1 = np.repeat(V1, N2).reshape(N1,N2)
        V2 = np.repeat(V2, N1).reshape(N2,N1).T
        return V1, V2

    def get_vals(self,v,t=0,V1=None,V2=None):
        if V1 is None:
            V1, V2 = self.get_V1V2_mat()
        coo = np.where(v > t)
        x = V1[coo].flatten()
        y = V2[coo].flatten()
        c = v[coo].flatten()
        return x,y,c

    def get_V1V2_conditioned_V3(self,param):
        r = self.get_plot_pool('compryd')
        gap = .02
        coo = np.where((param+gap > r) & (r > param-gap))
        x = r[:,0][coo]
        y = r[:,1][coo]
        return x,y

    def mf_ops_converged(self):
        mf_ops = np.array([self.outputdata['xiQAH']] + [self.outputdata['rho%d'%_] for _ in range(1,4)])
        mf_ops /= np.max(mf_ops)
        mf_ops[mf_ops==1.] = 0
        self.mf_ops_gap = np.max(mf_ops)
        mf_ops[mf_ops < self.convergence_thresh_mfops] = 0
        mf_ops_converged = not bool(np.sum(mf_ops))
        return mf_ops_converged

    def get_mf_ops(self, show_warnings=True):
        # Extract data for phase diagram plot.
        xiQAH = np.abs(self.datadict['xiQAH'].squeeze())
        V1V2gridshape = xiQAH.shape
        # mf_ops : tensor of order parameters.
        mf_ops = np.zeros((4,) + V1V2gridshape)
        mf_ops[0] = xiQAH
        for _ in range(1, 4):
            mf_ops[_] = self.datadict['rho'+str(_)].squeeze()
        # Set mf_ops to zero when below numeric threshold.
        mf_ops[mf_ops < self.convergence_thresh_mfops] = 0
        # Check if there is more than one mf_ops above threshold. It should not be the case. Print a warning.
        ms = np.sum(np.array(mf_ops, dtype=bool), axis=0)
        ms[ms==1] = 0
        cond = np.array_equal(ms, np.zeros(V1V2gridshape))
        if not cond:
            # Identify V1,V2 with multiple finitprinte op, set them all to 0 (invalidate data point).
            xs,ys = np.where(ms != 0)
            V1,V2 = self.get_V1V2_mat()
            self.permaprt('*'*300+'Beware! Data point with multiple finite order parameters!',verbose=show_warnings)
            for _ in range(len(xs)):
                x,y = xs[_],ys[_]
                self.permaprt('V1: %.2f, V2: %.2f, ' %(V1[x,y], V2[x,y]), mf_ops[:,x,y], verbose=show_warnings)
                mf_ops[:,x,y] *= 0
        return mf_ops

    def get_pd_ordparams(self,ax,markersize,show_warnings=True):
        mf_ops = self.get_mf_ops(show_warnings=show_warnings)
        if self.rydberg:
            for iV1 in range(len(self.V1s)):
                w = np.where(self.V2s > 511/574*self.V1s[iV1])
                mf_ops[:, iV1, w] *= 0
            mf_ops[:, :, np.where(self.V2s == 0)] *= 0
        # Draw scatter
        colors = ['Reds', 'Greens_r', 'Blues_r', 'spring']
        scs = []
        for _ in range(4):
            x, y, c = self.get_vals(mf_ops[_])
            sc = ax.scatter(x, y, c=c, cmap=colors[_], s=markersize, marker='s')
            scs.append(sc)
        return scs[0], ax

    def plot_phasediag_rydberg(self, markersize=60, figsize=4.5, fontsize=15):
        plt.close()

        # Define figure structure
        aspect_ratio = 1.2
        fig = plt.figure(figsize=[figsize*aspect_ratio,figsize])
        gs0 = mpl.gridspec.GridSpec(1,2, width_ratios=[100,10])
        ax1 = fig.add_subplot(gs0[0,0])
        ax2 = fig.add_subplot(gs0[0,1])

        # Params
        tickspad = 3
        cbarpad = 1
        spinelw = 1
        lw = 1

        # Order parameter scatter plot
        sc, ax1 = self.get_pd_ordparams(ax1,markersize,show_warnings=False)

        # Axes limits
        x0,x1,y0,y1 = self.V1V2region
        ax1.set_xlim(x0, x1)
        ax1.set_ylim(y0, y1)

        # Set ticks
        ax1.set_xticks(np.linspace(x0,x1,1+x1-x0))
        ax1.set_yticks(np.linspace(y0,y1,1+y1-y0))

        # Set tick labels
        ax1.set_xticklabels(np.linspace(x0,x1,1+x1-x0), fontsize=fontsize)
        ax1.set_yticklabels(np.linspace(y0,y1,1+y1-y0), fontsize=fontsize)

        # Tick params
        ax1.tick_params(axis="x", pad=tickspad, labelsize=fontsize)
        ax1.tick_params(axis="y", pad=tickspad, labelsize=fontsize)

        # Tick labels
        ax1.set_xlabel(r'$V_1/t$', fontsize=fontsize)
        ax1.set_ylabel(r'$V_2/t$', rotation=0, fontsize=fontsize)
        ax1.yaxis.set_label_coords(-1./figsize*aspect_ratio, .58)

        # Set spines
        for axis in ['top','bottom','left','right']:
            ax1.spines[axis].set_linewidth(spinelw)

        # Draw Rydberg potential boundaries
        a = 10
        ax1.plot([0, a], [0, a*511/574], '--', c='black', lw=lw)
        ax1.plot([0, a], [0, a/8], '--', c='black', lw=lw)

        # Colorbar
        ticks = [.04, .06, .08, .10]
        cbar = fig.colorbar(sc, ticks=ticks, cax=ax2, pad=cbarpad)
        cbar.ax.tick_params(which='major', length=0, width=0, labelsize=fontsize, pad=1)
        cbar.ax.set_title(r'$\xi_{QAH}$', fontsize=fontsize)

        # Finalize
        ax1.set_aspect(1)
        return plt

    def plot_phasediag_fixedV3(self, V3=0, markersize=60, figsize=3, fontsize=15):
        plt.close()

        # Define figure structure
        aspect_ratio = 1
        fig = plt.figure(figsize=[figsize*aspect_ratio,figsize])
        gs0 = mpl.gridspec.GridSpec(1,1)
        ax1 = fig.add_subplot(gs0[0,0])
        ax1.set_aspect(aspect_ratio)

        # Params
        tickspad = 3
        spinelw = 1

        # Order parameter scatter plot
        sc, ax1 = self.get_pd_ordparams(ax1,markersize,show_warnings=True)

        # Axes limits
        x0,x1,y0,y1 = self.V1V2region
        ax1.set_xlim(x0, x1)
        ax1.set_ylim(y0, y1)

        # Set ticks
        ax1.set_xticks(np.linspace(x0,x1,1+x1-x0))
        ax1.set_yticks(np.linspace(y0,y1,1+y1-y0))

        # Set tick labels
        ax1.set_xticklabels(np.linspace(x0,x1,1+x1-x0), fontsize=fontsize)
        ax1.set_yticklabels(np.linspace(y0,y1,1+y1-y0), fontsize=fontsize)

        # Tick params
        ax1.tick_params(axis="x", pad=tickspad, labelsize=fontsize)
        ax1.tick_params(axis="y", pad=tickspad, labelsize=fontsize)

        # Axes labels
        ax1.set_xlabel(r'$V_1/t$', fontsize=fontsize)
        ax1.set_ylabel(r'$V_2/t$', rotation=0 , fontsize=fontsize)
        ax1.yaxis.set_label_coords(-1./figsize*aspect_ratio, .58)

        # Set spines
        for axis in ['top','bottom','left','right']:
            ax1.spines[axis].set_linewidth(spinelw)
        return plt

    def get_Vsarray_pool(self):
        # Full V3 fixed
        dim = np.prod(self.Vsgridshape)
        Vsarray_pool = np.zeros([dim,self.nV], dtype=float)
        for _ in range(self.nV):
            nr = int(np.prod(self.Vsgridshape[_+1:]))
            nt = int(np.prod(self.Vsgridshape[:_]))
            Vsarray_pool[:,_] = np.tile(np.repeat(self.Vsdict[_],nr),nt)

        # Full pool, Rydberg potential
        def nan(v): v[v==0]=np.nan; return v
        r = np.copy(Vsarray_pool)
        V1 = r[:,1]
        V2 = r[:,2]
        r[:,3] = V1*V2 / (9*V1 - 8*V2)
        r[:,4] = 7*V1*V2 / (124*V1 - 117*V2)
        r, indexes = np.unique(r,axis=0,return_index=True)
        Vsarray_pool_ryd = np.zeros(r.shape)
        Vsarray_pool_ryd[indexes] = r

        # Compact pool, Rydberg potential
        r = np.copy(Vsarray_pool_ryd)
        for _ in range(1,self.nV):
            cond = np.where(r[:,_] == 0); rp = np.delete(r, cond, 0)
            cond = np.isnan(r[:,_]);      r = np.delete(r, cond, 0)
        cond = np.where(r[:,2] >= r[:,1]);   r = np.delete(r, cond, 0)
        cond = np.where(r[:,1] >= 8*r[:,2]); r = np.delete(r, cond, 0)
        compact_Vsarray_pool_ryd = r
        return Vsarray_pool, Vsarray_pool_ryd, compact_Vsarray_pool_ryd 

    def get_V1V2s(self):
        V1s = np.linspace(self.V1V2region[0], self.V1V2region[1], self.Vsgridshape[1])
        V2s = np.linspace(self.V1V2region[2], self.V1V2region[3], self.Vsgridshape[2])
        return V1s, V2s

    def permaprt(self,*message,verbose=True):
        # Pr*nt function with a weird enough name, to make it easy to find "pr*nt()" statements dropped around.
        if verbose: print(*message)
        return

Perform the mean-field analysis.

Methods
----
| Method  | Description |
|----|----|
|`__init__` |  Initialize Hamiltonian related parameters, then call checkerboard_eight_sites.__init__()  |
|`prepare_run`|  Update the set of $V_i$, define Hamiltonian accordingly, initialize random correlation matrix, set and initialize other run parameters like k-space eigen-vectors and eigen-values (bands)  |
